# Boids
Gaëtan Dufay & Maximilien Escalera

Nous tentons d'étudier ici le comportement grégaire[$^{[1]}$](#Sources) des animaux observé dans la nature. Ce comportement est présent dans un vol d'hirondelles, un banc de saumons, un troupeau de bisons... Il émerge quand un grand nombre d'animaux agissent ensemble donnant l'impression d'une volonté de groupe :

> the strong impression of intentional centralized control
>
> Reynolds, 1987[$^{[2]}$](#Sources).


Il semblerait que ces comportements se soient développés comme un mécanisme de défense contre les prédateurs. En effet, être en groupe confère l'avantage d'être plus alerte à l'arrivée d'un prédateur même si il procure le désavantage d'attirer lesdits prédateurs[$^{[3]}$](#Sources).

Nous allons modéliser ce comportement à travers une simulation de boids. Le modèle des boids est un modèle simple développé par Craig W. Reynolds en 1986[$^{[4]}$](#Sources), qui vise à recréer les comportements grégaires observés dans la nature, le terme boid (contraction de bird-oid) désignant la créature générique composant la foule simulée. Chaque boid individuel ne communique pas avec ses voisins mais fait seulement des choix sur sa trajectoire en fonction de son voisinage.

Ces choix sont effectués en suivant trois règles simples :
- **Séparation** : les boids ajustent leur trajectoire afin de ne pas se collisionner entre eux
- **Alignement** : les boids s'orientent de manière à s'aligner avec leur voisinage
- **Cohésion** : pour former un groupe, les boids se rapprochent du centre de masse de leur voisinage

Il faut alors définir le *voisinage* d'un boid. Nous considérons qu'un boid se trouve dans le voisinage d'un autre boid s'il est dans son champ de vision, à savoir une sphère a laquelle nous retirons un angle solide, correspondant à l'angle mort du boid.
Ainsi, on peut se représenter un boid comme suit en deux dimensions:

<figure>
    <img src="https://upload.wikimedia.org/wikipedia/commons/1/17/Boids.png" width="256">
    <figcaption>Voisinnage d'un boid (Ænthaüs, CC BY-SA 4.0 via Wikimedia Commons)</figcaption>
</figure>

Du rouge le plus foncé au plus clair: zone de répulsion, d'orientation et d'attraction. La partie grise est hors du champ de vision.
Dans notre cas, les disques sont en faites des sphères car nous faisons la simulation en 3D.

A ces trois règles principales, s'ajoutent d'autres règles pour rendre le comportement plus réaliste (limitation de la vitesse, évolution dans un espace restreint...).[$^{[4][5][6]}$](#Sources)

On cherche alors à étudier les caractéristiques du ou des groupes ainsi formés.

In [ ]:
import numpy as np # calcul vectoriel
import matplotlib.pyplot as plt # visualisation et graphismes
from mpl_toolkits.mplot3d import axes3d  # Fonction pour la 3D
import matplotlib.animation as animation # animation
import itertools as it # générateurs
from collections import deque # double liste chainée
from ipywidgets import IntProgress # barre de progression
from IPython.display import display # affichage pour ipywidgets
import joypy # joyplot

## Table des matières

- [Algorithme](#Algorithme)
    - [Spatial hash grid](#Spatial-hash-grid)
    - [Classe de vecteurs en trois dimensions](#Vec3)
    - [Boid](#Boid)
    - [Statistiques](#Statistiques)
- [Un premier groupe](#Un-premier-groupe)
- [Poissons](#Poissons)
    - Visualisation
    - Statistiques
- [Oiseaux](#Oiseaux)
    - Visualisation
    - Statistiques
- [Essaim](#Essaim)
    - Visualisation
    - Statistiques

## Algorithme

### Spatial hash grid

Une implémentation naïve de l'algorithme pour calculer la nouvelle vitesse de chaque individu consiste à itérer sur tous les boids pour chaque boid. Cela nous donne donc une compléxité en $O(n^2)$, c'est-à-dire que si nous doublons le nombre de boids par exemple, nous multiplions le temps de calcul par quatre. Or, comme il n'est pas rare d'avoir plusieurs centaines (voir milliers) d'individus au sein d'une nuée d'oiseaux ou d'un banc de poisson[$^{[7]}$](#Sources), il est nécessaire d'implémenter un algorithme plus "intelligent".

Pour ce faire, on décide d'implémenter un spatial hash.
Un spatial hash est une manière d'indexer des objets dans l'espace[$^{[8]}$](#Sources) et nous permet de reduire la complexité à $O(n \log(n))$. Le principe est de découper l'espace (2D ou 3D) en un ensemble de cellule de taille fixes, chaque cellule pouvant contenir un nombre variables d'objets (ici des boids). Cela nous permet ainsi d'itérer seulement sur les cellules proches d'un boid donc d'ignorer des boids qui n'auraient de toutes façons pas été dans le champ de vision, et ainsi de limiter le nombre d'itérations.

Le choix d'utiliser cet algorithme au lieu d'autres potentiellement plus rapides (par exemple octree) est notamment du à sa simplicité d'implémentation en python.

Cependant, et comme le souligne l'auteur[$^{[2]}$](#Sources), cette complexité est loin de représenter la réalité. Un groupe n'ayant à priori pas de limite à sa taille, la quantité de boid ne devrait pas impacter le temps de calcul des trajectoires, comme une sorte de grandeurs intensives du système.

In [ ]:
# sources :
# - https://www.pygame.org/wiki/SpatialHashMap
# - http://mauveweb.co.uk/posts/2011/05/introduction-to-spatial-hashes.html
#
# Modifications par rapport à la source :
# - ajout de la méthode from_boids dans la classe
# - utilisation de numpy pour les clés
# - utilisation d'un set plutôt que d'une liste
# - utilisation des fonctions par défaut
# - possibilités de supprimer / mettre à jour un élément
# - recherche dans des cases alentours

class HashMap(object):
    """
    Hashmap est un indexage spatial utilisé pour la détection de collision.
    """
    def __init__(self, cell_size=15):
        self.cell_size = cell_size # taille d'une cellule
        self.grid = {} # ensemble des cellules

    @classmethod
    def from_boids(cls, boids, cell_size=15):
        """
        Crée un HashMap à partir d'une liste de boids.
        """
        hashmap = cls(cell_size) # on instancie la class
        setdefault = hashmap.grid.setdefault
        key = hashmap.key

        # ajoute chaque boid dans la grille
        for boid in boids:
            setdefault(key(boid.position_), set()).add(boid)

        return hashmap


    def key(self, point):
        """
        Fonction de hashage.
        """
        cell_size = self.cell_size

        return tuple((np.floor(point / self.cell_size) * self.cell_size).astype(int))

    def insert(self, point, obj):
        """
        Insère un point dans la hashmap.
        """
        self.grid.setdefault(self.key(point), set()).add(obj)
        
    def remove(self, point, obj):
        """
        Supprime un point de la hashmap.
        """
        cell = self.query(self.key(point))
        cell.discard(obj)
        
    def update(self, old, new, obj):
        """
        Met à jour un point.
        """
        self.remove(old, obj)
        self.insert(new, obj)

    def query(self, point):
        """
        Retourne tous les objets dans la cellule point.
        """
        return self.grid.get(point, set())
    
    def find_near(self, point):
        """
        Retourne tous les objets des cellules adjacentes à un point.
        """
        ret = set()
        radius = self.cell_size
        x, y, z = self.key(point)

        # on multiplie par 2 car la dernière cellule n'est pas incluse
        xmin, xmax = x - radius, x + 2 * radius
        ymin, ymax = y - radius, y + 2 * radius
        zmin, zmax = z - radius, z + 2 * radius

        # ensemble des cellules adjacentes
        indexes = it.product(range(xmin, xmax, self.cell_size), range(ymin, ymax, self.cell_size), range(zmin, zmax, self.cell_size))

        # on regarde dans toutes les cellules adjacentes
        for idx in indexes:
            ret |= self.query(idx) # ajout des objets non présent

        return ret

### Vec3

In [ ]:
# sources :
# - https://numpy.org/doc/stable/user/basics.subclassing.html#slightly-more-realistic-example-attribute-added-to-existing-array

class Vec3(np.ndarray):
    """
    Sous-classe de numpy.ndarray représentant un vecteur de dimension 3.
    On y a ajouté quelques méthodes utiles.
    """
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)

        return obj
    
    def norm(self):
        """Retourne la norme du vecteur"""
        return np.linalg.norm(self)
    
    def cos_angle(self, other):
        """Retourne le cosinus de l'angle formé entre ce vecteur et un autre"""
        normed = self / self.norm()
        o_normed = other / other.norm()
        
        return normed.dot(o_normed)

### Boid[$^{[5]}$](#Sources)

Comme dit plus haut, la créature générique qu'on cherche à simuler est un boid.  
En plus des 3 règles de base (**cohésion**, **alignement** et **séparation**), nous avons décidé de rajouter :

- une limitation de la vitesse, pour paraître plus réaliste
- une limitation de l'espace, pour faire naitre plus d'interactions entre les boids
- une mémoire de la trajectoire, pour analyser plus en détails les déplacements par la suite

Une des limites de notre modélisation est que nous n'avons pas défini d'unité temporelle ni spatiale. Ainsi, le boid se déplace d'une certaine direction pour un certain pas de temps, mais sans référence particulière. 

In [ ]:
# sources :
# - http://www.kfish.org/boids/pseudocode.html
#
# Modifications :
# - code en python
# - classe
# - ajout du champ de vision, vitesse limite, zones de perception
# - sauvegarde de l'état courant

class Boid:
    """
    Représentation d'un boid.
    """
    def __init__(self, fov=15*np.pi/16, attraction=150., repulsion=5., orientation=150., coherence_factor=.5,
                 repulsion_factor=1, align_factor=.125, vmax=10., vmin=5., v_init=[3., 3., 3.],
                 bounds=200, bound_reaction=10., spawn_size=50):
        self.fov_ = fov # champ de vision, en rad
        
        # on instancie le boid à une position aléatoire
        # la densité initiale sera plus ou moins importante selon spawn_size
        self.position_ = Vec3(np.random.random_sample(3)) * spawn_size - (spawn_size / 2)
        # tous les boids commencent avec la même vitesse initiale
        self.velocity_ = Vec3(np.array(v_init))
        
        # taille des voisinnages pour les 3 règles
        self.attraction_, self.repulsion_, self.orientation_ = attraction, repulsion, orientation
        
        # facteurs d'importance des trois règles
        self.coherence_factor_ = coherence_factor
        self.repulsion_factor_ = repulsion_factor
        self.align_factor_ = align_factor
        
        self.vmax_, self.vmin_ = vmax, vmin # vitesses limites
        self.bounds_ = bounds # espace limite, [-bounds, bounds] pour chaque axe
        self.bound_reaction_ = bound_reaction # force de rappel des limites du monde
        self.history_ = deque() # historique de position et vitesse
        
        self.save_state() # on sauvegarde les paramètres initiaux
        
    def filter_boids(self, boids, radius):
        """
        Retourne les boids présent dans un certain radius et dans le champ de vision.
        """
        # filtre par rapport au champ de vision
        filter_angle = lambda other: self.velocity_.cos_angle(other.position_ - self.position_) > np.cos(self.fov_)
        # filtre par rapport radius
        filter_pos = lambda other: (other.position_ - self.position_).norm() < radius

        return filter(filter_angle, filter(filter_pos, boids))

    def rule_1(self, boids):
        """Règle 1 : cohésion : pour former un groupe, les boids se rapprochent les uns des autres"""
        c = Vec3(np.zeros(3))
        filtered_pos = [boid.position_ for boid in self.filter_boids(boids, self.attraction_)]

        # on vérifie qu'il y a bien des boids
        # sinon on ne se rapproche pas
        if len(filtered_pos) >= 1:
            c = sum(filtered_pos) / len(filtered_pos)
        else:
            c = self.position_

        return (c - self.position_) * self.coherence_factor_

    def rule_2(self, boids):
        """Règle 2 : séparation : 2 boids ne peuvent pas se trouver au même endroit au même moment"""
        c = Vec3(np.zeros(3))

        for boid in self.filter_boids(boids, self.repulsion_):
            c += (self.position_ - boid.position_) * self.repulsion_factor_

        return c

    def rule_3(self, boids):
        """Règle 3 : alignement : pour rester groupés, les boids essayent de suivre un même chemin"""
        c = Vec3(np.zeros(3))
        filtered_vel = [boid.velocity_ for boid in self.filter_boids(boids, self.orientation_)]

        # on vérifie qu'il y a bien des boids
        # sinon on ne s'aligne pas
        if len(filtered_vel) >= 1:
            c = sum(filtered_vel) / len(filtered_vel)
        else:
            c = self.velocity_

        return self.velocity_ + c * self.align_factor_

    def limit_velocity(self):
        """Limitation de la vitesse du boid pour paraître plus réaliste"""
        norm = self.velocity_.norm()

        # on réduit la vitesse si elle trop grande
        if norm > self.vmax_:
            self.velocity_ = self.velocity_ / norm * self.vmax_
        # on augment la vitesse si elle trop petite
        if norm < self.vmin_:
            self.velocity_ = self.velocity_ / norm * self.vmin_

    def bound_position(self):
        """Limite la position dans l'espace"""
        v = Vec3(np.zeros(3))
        posx, posy, posz = self.position_

        # on regarde si la position est en dehors des limites de l'espace
        # si c'est le cas, on change la direction
        if posx < -self.bounds_: v[0] += self.bound_reaction_
        if posx > self.bounds_: v[0] -= self.bound_reaction_
        if posy < -self.bounds_: v[1] += self.bound_reaction_
        if posy > self.bounds_: v[1] -= self.bound_reaction_
        if posz < -self.bounds_: v[2] += self.bound_reaction_
        if posz > self.bounds_: v[2] -= self.bound_reaction_
        
        return v
    
    def save_state(self):
        """Sauvegarde l'état courant du boid"""
        # on fait une copy pour éviter les changements de mémoire
        p_copy, v_copy = self.position_.copy(), self.velocity_.copy()

        self.history_.append((p_copy, v_copy))
        
    def rewind(self, t=0):
        """
        Retourne à l'instant t en arrière.
        Si t n'est pas donné, retourne l'origine des temps.
        """
        self.position_, self.velocity_ = self.history_[t]

Maintenant que l'on a des boids, on veut pouvoir simuler un groupe (troupeau, nuée, banc, ...) :

In [ ]:
class Boids:
    """Classe container pour l'ensemble des boids"""

    def __init__(self, nb_boids, cell_size, **kwargs):
        self.boids = [Boid(**kwargs) for _ in range(nb_boids)] # ensemble des boids
        self.world = HashMap.from_boids(self.boids, cell_size) # monde dans lequel les boids évoluent
        
    def __len__(self):
        """Retourne le nombre de boid"""
        return len(self.boids)

    def move_boids(self):
        """
        Calcul de la nouvelle position des boids.
        """
        for boid in self.boids:
            old_pos = boid.position_.copy() # on sauvegarde la position courante
            boids = self.world.find_near(boid.position_) # on recupère tous les boids d'une même cellule
            boids.remove(boid) # on supprime le boid pour ne pas qu'il se compte lui-même

            v1 = boid.rule_1(boids)
            v2 = boid.rule_2(boids)
            v3 = boid.rule_3(boids)
            v4 = boid.bound_position()

            boid.velocity_ += v1 + v2 + v3 + v4
            boid.limit_velocity()
            boid.position_ += boid.velocity_ # déplacement instanané de la position

            self.world.update(old_pos, boid.position_, boid) # on sauvegarde la nouvelle position dans la hashmap
            boid.save_state()
            
    def get_history(self, index=None):
        """
        Retourne l'historique des positions et vitesses pour un ou tous les boids
        """
        if index is None:
            return [boid.history_ for boid in self.boids]
        
        return self.boids[index].history_
    
    def rewind(self, t=0):
        """
        Retourne à l'instant t.
        Si t n'est pas donné, on retourne à l'origine.
        """
        for boid in self.boids:
            boid.rewind(t)

    def plot(self, num_steps, bounds=110, title="Trajectoire des Boids"):
        """
        Visualise la trajectoire des boids au cours du temps.
        """
        def update_lines(num, boids, positions, directions):
            """Fonction de visualisation 3D des trajectoires des boids"""
            for boid, position, direction in zip(boids, positions, directions):
                pos, vel = boid[num] # on recupère les position et vitesse num
                d = np.array([pos, pos + vel]) # la direction est donnée par la vitesse
        
                position.set_data_3d(pos)
                direction.set_data_3d(d.T)

            return positions, directions


        plt.rcParams["figure.figsize"] = (9, 6) # taille des graphes
        plt.rcParams["font.size"] = 16 # taille police graphes
        plt.rcParams["axes.grid"] = True # affiche automatiquement la grille

        history = self.get_history() # on récupère les informations de la trajectoire

        # Tracé du résultat en 3D
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")

        ax.set_title(title)
        ax.set(xlim3d=(-bounds,bounds), xlabel='X')
        ax.set(ylim3d=(-bounds,bounds), ylabel='Y')
        ax.set(zlim3d=(-bounds,bounds), zlabel='Z')

        positions = [ax.plot([], [], [], "ok")[0] for i in range(self.__len__())]
        directions = [ax.plot([], [], [], "-r")[0] for i in range(self.__len__())]

        # Creation de l'animation
        return animation.FuncAnimation(
            fig, update_lines, frames=num_steps, fargs=(history, positions, directions), interval=50, repeat=False, blit=True)

Il est important de bien choisir la taille des cellules de hashmap pour permettre d'optimiser le code tout en prenant en compte tous les boids dans le voisinage. Si nous prenons la taille de la cellule égale au rayon de la plus grande sphère de vision, nous garantissons que nous ne ratons aucun boid, même si le boid pour lequel nous faisons les calculs se trouve à la frontière entre deux cellules.
Ici nous la posons égale au rayon de la sphère d'attraction ou d'orientation qui ont la même taille.

In [ ]:
#paramètres généraux
num_steps = 200 # durée de la simulation
N = 50 # nombre de boids
cell_size = 150 # taille d'une cellule de hashmap

def calcul(boids, num_steps=num_steps):
    """Effectue les calculs pour chaque pas de temps et affiche une barre de progression"""
    f = IntProgress(description="Calcul...", min=0, max=num_steps) # barre de progression
    display(f) # affichage de la barre

    # calcul des positions au cours du temps
    for _ in range(num_steps):
        boids.move_boids()

        f.value += 1 # màj barre
    
    f.description = "Ok !" # calcul des positions fini

On a désormais tous les éléments nécessaires pour simuler un comportement de groupe. Nous allons maintenant développer des outils afin d'étudier l'émergence de comportements :

### Statistiques

In [ ]:
class Stats:
    """
    Classe pour réaliser différentes statistiques
    """
    def __init__(self, positions, velocities, name, num_steps=num_steps):
        self.positions = positions
        self.velocities = velocities
        self.name = name # nom à afficher sur les graphes
        self.num_steps = num_steps # nombre de pas de temps
        self.steps = np.array(list(range(num_steps + 1))) # liste des pas de temps


    @classmethod
    def from_history(cls, history, *args):
        """Change la forme de l'historique pour pouvoir faire des statistiques"""
        reshape = np.array([
            [(p, v) for p, v in boid] for boid in history]
        )
        
        # on groupe par pas de temps
        reshape = np.array(
            [reshape[:, i] for i in range(len(history[0]))]
        )
        pos, vel = reshape[:,:,0], reshape[:,:,1]

        return cls(pos, vel, *args)
    
    def alignement(self):
        """renvoie la moyenne des valeurs absolues des produits scalaires des vitesses normées à chaque pas de temps"""
        moy_angles = [0 for _ in range(self.num_steps + 1)]

        for t in range(self.num_steps + 1):
            angle = 0
            cmp = len(self.positions[0]) * (len(self.positions[0]) - 1) / 2

            # pour chaque boid, on compare l'alignement aux autres
            for i in range(len(self.positions[0])):
                for j in range(i):
                    angle += self.velocities[t, i].view(Vec3).cos_angle(self.velocities[t, j].view(Vec3))
            
            moy_angles[t] = angle / cmp

        return moy_angles

    def etalement(self):
        """renvoie la moyenne de la distance au centre de masse pour chaque boid par pas de temps"""
        moy_etalement = [
            np.mean(np.linalg.norm(position.mean(axis=0) - position, axis=1)) for position in self.positions
        ]

        return moy_etalement

    def barycenter(self):
        """Calcule la norme de la position du barycentre pour chaque pas de temps"""
        barycenters = self.positions.mean(axis=1)

        return np.linalg.norm(barycenters, axis=1)

    def vel_barycenter(self):
        """Calcule la norme de la vitesse du barycentre pour chaque pas de temps"""
        barycenters = self.velocities.mean(axis=1)

        return np.linalg.norm(barycenters, axis=1)

    def accel_barycenter(self):
        """Calcule l'évolution de la vitesse du barycentre entre chaque pas de temps donc son accéleration"""
        barycenters = self.velocities.mean(axis=1)
        barycenters = np.diff(barycenters, axis=0)

        return np.linalg.norm(barycenters, axis=1)

    # Documentation du module joyplot :
    # https://deepnote.com/@deepnote/Joyplot-Introduction-4666e1a3-3249-442e-9a94-2bbcc5cb1b1d
    def distribution(self, t):
        """Renvoie la distribution moyenne des boids au temps t"""
        
        nb_boids = len(self.positions[0]) #correspond au nombre de boids
        distribs = np.zeros(nb_boids)
    
        for i in range(nb_boids):
            center_pos = self.positions[t, i] #on va calculer la distribution autour d'un boid

            distribs = distribs + np.linalg.norm(self.positions[t] - center_pos, axis=1)

        #nous introduisons un 0 dans la somme des distances aux autres boids (distance avec lui même)
        #donc nous divisons par le nombre de boids - 1 pour ne pas comptabiliser cette valeur dans la moyenne    
        return distribs / (nb_boids - 1)
    
    def plot(self):
        """Renvoie des graphes présentant toutes les données analysées par les fonctions précédentes"""
        plt.rcParams["figure.figsize"] = (16, 10) # taille des graphes
        plt.rcParams["figure.autolayout"] = True # ajustement des subplots
        plt.rcParams["axes.grid"] = True # affiche automatiquement la grille

        fig = plt.figure()

        # Étalement des boids au cours du temps
        fig.add_subplot(521)
        plt.plot(self.steps, self.etalement(), "+")
        plt.xlabel("Temps")
        plt.ylabel("Distance moyenne au centre de masse")
        plt.title(f"Etalement des '{self.name}' au cours du temps")

        # Alignement des boids au cours du temps
        fig.add_subplot(522)
        plt.plot(self.steps, self.alignement(), "+")
        plt.ylim(-0.01, 1.01)
        plt.xlabel("Temps")
        plt.ylabel("anlge moyen entre tous les boids")
        plt.title(f"Alignement des '{self.name}' au cours du temps")

        # Norme de la position du barycentre au cours du temps
        fig.add_subplot(5, 2, (3, 4))
        plt.plot(self.steps, self.barycenter(), "+")
        plt.xlabel("Temps")
        plt.ylabel("Barycentre")
        plt.title(f"Norme du barycentre des '{self.name}' au cours du temps")

        # Vitesse du barycentre au cours du temps
        fig.add_subplot(5, 2, (5, 6))
        plt.plot(self.steps, self.vel_barycenter(), "+")
        plt.xlabel("Temps")
        plt.ylabel("Vitesse du barycentre")
        plt.title(f"Norme de la vitesse du barycentre des '{self.name}' au cours du temps")

        # Accélération du barycentre au cours du temps
        fig.add_subplot(5, 2, (7, 8))
        plt.plot(self.steps[1:], self.accel_barycenter(), "+")
        plt.xlabel("Temps")
        plt.ylabel("Accélération du barycentre")
        plt.title(f"Accélération du barycentre des '{self.name}' au cours du temps")

        plt.show()
        
        plt.rcParams["figure.autolayout"] = False # plus d'ajustement

        #Distribution moyenne des boids au cours du temps
        distrib = [self.distribution(t) for t in range(self.num_steps)]
        labels = [i if i % 10 == 0 else None for i in range(self.num_steps)]
        fig, ax = joypy.joyplot(distrib, range_style='own', ylim="own",
                                  grid="y", linewidth=1, legend=False, figsize=(12,8),
                                  title=f"Evolution de la distribution des {self.name}", labels=labels)
        ax[-1].yaxis.set_label_position("right")
        ax[-1].yaxis.set_visible(True)
        ax[-1].yaxis.set_ticks([])
        ax[-1].set_ylabel("Temps")
        ax[-1].set_xlabel("Distance moyenne entre les boids")

        plt.show()

### Un premier groupe

Maintenant que nous avons tous les outils nécessaires, nous pouvons tenter de simuler puis d'analyser l'émergence d'un comportement de groupe.

In [ ]:
# on crée une instance de la classe avec 100 boids
boids = Boids(N, 100)

# on calcule les positions de tous les boid pour chaque pas de temps (ici 200 par défaut)
calcul(boids)

In [ ]:
# matplotlib intéractif
%matplotlib notebook

anim = boids.plot(num_steps, bounds=250, title="Trajectoire des boids")

Cette simulation à été effectuée avec les paramètres par défauts passés dans la classe Boid à savoir :
- champ de vision : $\frac{15\pi}{16}$ rad


- rayon des sphères :
    - d'attraction : 150
    - de repulsion : 5
    - d'orientation : 150


- facteurs :
    - d'attraction : 0.5
    - de repulsion : 1
    - d'orientation : 0.125


- limitation de la vitesse :
    - minimale : 5
    - maximale : 10

- vitese initiale : [3, 3, 3]


- taille de la boite : 200


- reaction aux limites : 10


- spawn size : 50

Le rayon des sphères affecte la distance à laquelle un boid va prendre en compte un autre boid pour appliquer chacune des trois règles, et cette distance peut varier d'une règle à l'autre. Les facteurs affectent à quel point un boid accorde de "l'importance" à une certaine règle. La taille de la boite est la taille de l'espace dans lequel sont libres d'évoluer les boids, tandis que la réaction aux limites est analogue aux facteurs vus plus haut si nous considérons une quatrième règle incitant les boids à rester dans la boite. Enfin, le paramètre *spawn_size* dicte la taille de l'espace dans lequel nous initialisons les positions des boids.

Nous voyons que les boids commencent en formant un seul groupe ce qui est attendu car ils ont tous la même vitesse initiale. Mais ce groupe peut se scinder en plus petits groupes qui suivent chacun leur trajectoire. En effet si le groupe est légèrement perturbé, des grands facteurs d'alignement et de cohérence vont avoir tendance à rapprocher les individus qui ont été séparés et engendrer la création de plus petits groupes.

Dans cette configuration, un trop grand groupe est instable et la configuration qui apparait naturellement est de plus petits groupes se détachant du groupe initial. Mais si nous augmentons la taille des limites, ainsi que du paramètre spawn_size afin d'initialiser la position des boids partout dans notre boite, que se passerait-il ? Nous ne commencerions plus avec un grand groupe initial, mais des groupes émergeraient-ils quand même ? Si oui, auraient-ils la même taille ?

In [ ]:
boids2 = Boids(N, 100, v_init=[0.1, 0.1, 0.1], bounds=400, spawn_size=400)
calcul(boids2)

In [ ]:
anim = boids2.plot(num_steps, bounds=450, title="Formation de groupes")

Les boids initialement répartis uniformément dans tout l'espace commencent a former de petits groupes similaires à ceux observés pour le premier cas de figure. Nous voyons néanmoins que les boids ont plus de mal à former des groupes et à la fin de la simulation, ils sont un peu plus éparpillés.

Il semble que les boids avec ces paramètres aient une configuration stable qui émerge naturellement quand nous les laissons évoluer dans l'espace que nous leur donnons. Nous pouvons également analyser les données à l'aide des fonctions que nous avons créées :

In [ ]:
# plus besoin du mode interactif
%matplotlib inline

stats_boids = Stats.from_history(boids.get_history(), "boids")
stats_boids.plot() #cela peu prendre un peu de temps, surtout pour le dernier graphe 'distribution moyenne'

Ce que nous avions vu au sujet des groupes se scindant en groupes plus petits se voit dans l'étalement qui croit au cours du temps. Une baisse de l'étalement peut être du à la fusion de deux groupes qui se croisent comme nous avons également pu observer dans l'animation.

Il peut être intéressant de remarquer que l'alignement des boids et la norme de la vitesse du barycentre des boids semblent être très fortement corrélés. Ceci est attendu, car si l'alignement est élevé, tous les boids vont dans le même sens et le barycentre avance avec le reste des boids à cette vitesse là. Si au contraire l'alignement est faible, les boids vont dans tous les sens et nous pouvons imaginer le barycentre qui reste sur place.

La norme du barycentre des boids semble évoluer avec une certaine périodicité, ceci s'explique par la barières que nous avons imposées, les boids commencent à aller dans une certaine direction, puis en rencontrant la barrière repartent plus ou moins dans la direction opposée et ainsi de suite. Ceci se voit bien également dans l'animation. Le temps entre deux maxima semble être autour de 30 pas de temps, et la boite dans laquelle évoluent les boids fait 250 unité de longeur : $\frac{250}{30} \approx 8.33$ cette vitesse est plausible, elle est entre les bornes maximles de 5 et 10.

Comme nous venons d'établir, la norme du barycentre semble évoluer de manière périodique, et à chaque maxima dans cette évolution, semble correspondre une baisse temporaire de la vitesse du barycentre et un pic dans son accéleration. Comme vu précédement, les maxima de la norme du barycentre devraient correspondre au moment ou les boids atteignent la barrière. Il n'est donc pas étonnant de voir à ces moments là, une baisse de la vitesse du barycentre car les boids font demi tour. Par définition, un changement de vitesse entrainera un pic d'accélération. Il ne se passe rien au niveau des minima de la norme du barycentre car ceci correspond juste au moment où les boids passent par l'origine.

Enfin, l'évolution de la distribution moyenne des boids donne toujours la même idée que les boids s'étalent au cours du temps avec l'étalement des pics que l'on voit. Mais elle nous permet également de voir la formation de nouveaux groupes sous la forme de nouveaux pics se séparant et s'éloignant du pic principal. En effet si tous les boids sont dans un seul groupe, nous verrons un seul  grand pic centré sur la distance moyenne entre deux boids qui sera plus ou moins étalé selon si les boids eux mêmes sont très étalés ou non. Maintenant, si un groupe se sépare et se situe à une distance $d$ du groupe principal, nous verrons toujours un même pic centré sur la distance moyenne entre deux boids du même groupe, mais également un pic secondaire, plus petit, centré sur $d$ qui sera la distance moyenne entre deux boids qui ne sont pas dans le même groupe. Le pic principal en plus de s'étaler avance sur l'axe des abscisses, ce qui nous indique que les boids auront tendance à être de plus en plus éloignés les uns des autres avec le temps.

## Poissons

On cherche à faire émerger une structure de type banc de poisson.  
Ce type de groupe à la particularité de rester très groupé ce qui a pour effet de rendre les temps de calcul long, l'utilisation du spatial hash n'aidant pas vraiment. Pour cela, on donne des grands coefficients de répulsion et d'alignement et un facteur de cohésion plus faible. Chaque taille de zone de voisinage est différente avec une zone de repulsion relativement faible par rapport aux autres pour rendre le tout bien dense.

In [ ]:
poissons = Boids(N, cell_size=30, fov=15*np.pi/16, attraction=60., repulsion=10., orientation=50.,
                 coherence_factor=.07, repulsion_factor=.3, align_factor=.7, vmin=1, vmax=5,
                 bounds=100, bound_reaction=1, spawn_size=50)

calcul(poissons)

### Visualisation

In [ ]:
# matplotlib intéractif
%matplotlib notebook

anim = poissons.plot(num_steps, bounds=150, title="Trajectoire des poissons")

Les boids restent bien groupés.

### Statistiques

In [ ]:
# plus besoin du mode interactif
%matplotlib inline

stats_poissons = Stats.from_history(poissons.get_history(), "poissons")
stats_poissons.plot()

On voit bien que l'alignement reste constant et proche de 1. L'étalement est lui aussi relativement constant avec une distance au centre de masse qui est environ le double de la taille de la zone de répulsion. La vitesse du groupe est constante aussi.  
Ces résultats paraissent cohérent car pour que le groupe reste uni il faut qu'il se déplace à la même vitesse et dans la même direction. La stabilité de l'étalement peut être perçu comme le fait que chaque poisson a trouvé sa place, ils ne rentrent donc pas en collision entre eux et ne perturbent ainsi pas la structure globale.  
On voit que l'accélération et le barycentre des poissons suivent la même tendance : les pics correspondent aux réactions des bords du volume (l'amplitude des pics étant toujours positifs car on compare les normes des vecteurs). C'est encore une fois cohérent car le banc étant groupé, son barycentre suit son mouvement globale, l'accélération apparaissant lors du changement de direction dû aux bords.  
Enfin, on voit que la distribution des poissons évolue très peu au fil du temps, appuyant ainsi plus les résultats précédents.

## Oiseaux [$^{[9]}$](#Sources)

On essaie maintenant pour une structure de nuée d'oiseaux.  
Bien que ressemblant au banc de poissons, une nuée d'oiseaux a des individus plus éloignés les uns des autres et ces derniers peuvent potentiellement se séparer en plus petits groupes un court instant.

On va comparer pour deux structures avec des paramètres initiaux différents :

1. Un petit voisinage, dont la taille des zones de perception sont identiques, avec des facteurs de répulsion et d'alignement égaux, un faible facteur de cohérence. Le volume disponible de déplacement est plutôt petit et la densité initiale est faible.
2. On augmente la taille zones de perception qui ne sont plus identiques, le volume est plus grand, les facteurs sont tous différents et les boids sont beaucoup plus concentrés initialement.

In [ ]:
oiseaux_1 = Boids(N, cell_size=2, fov=15*np.pi/8, attraction=2., repulsion=2., orientation=2.,
                  coherence_factor=.5e-4, repulsion_factor=.05, align_factor=.05, vmax=3, vmin=2,
                  bounds=50, bound_reaction=.4, spawn_size=25)

# calcul plus long
oiseaux_2 = Boids(N, cell_size=60, fov=15*np.pi/16, attraction=60., repulsion=10., orientation=50.,
                  coherence_factor=.007, repulsion_factor=.03, align_factor=.07, vmax=6, vmin=3,
                  bounds=100, bound_reaction=1, spawn_size=10)

calcul(oiseaux_1)
calcul(oiseaux_2)

### Visualisation

In [ ]:
# matplotlib intéractif
%matplotlib notebook

anim = oiseaux_1.plot(num_steps, 100)

In [ ]:
anim = oiseaux_2.plot(num_steps, 150)

Cela ressemble assez à ce qu'on peut voir lorsqu'une nuée d'oiseaux vole.

### Statistiques

In [ ]:
# plus besoin du mode interactif
%matplotlib inline

stats_oiseaux_1 = Stats.from_history(oiseaux_1.get_history(), "oiseaux 1")
stats_oiseaux_2 = Stats.from_history(oiseaux_2.get_history(), "oiseaux 2")

stats_oiseaux_1.plot()

Contrairement aux poissons, l'étalement des oiseaux semblent plus disparates. On pouvait s'y attendre car comme la nuée peut se subdiviser en sous-groupe, la position du centre de masse fluctue lors de l'apparition de nouveaux groupes. On peut aussi voir le petit pic vers zéro vers $t = 35$ qui correspond aux regroupements des boids avant leur éclatement dans la visualisation.  
Encore une fois, les pics présents dans les autres graphes correspondent aux réactions avec les limites du monde. Si ici l'alignement et la vitesse présentent aussi des pics, c'est car les oiseaux ne forment pas une masse groupée comme les poissons, mais plutôt une ligne, donc lorsque les premiers oiseaux atteignent les murs et changent de direction, les autres eux continuent dans la même direction : il y a donc autant d'oiseaux dans un sens que dans l'autre d'où cette annulation de l'alignement et de la vitesse.  
Pour la norme du barycentre ainsi que de l'accélération, il s'agit de la même chose que pour les poissons : certes les oiseaux sont un peu plus distants les uns des autres, mais ils forment tout de même un groupe assez compact qui traduit le suivi du mouvement du groupe.  
Enfin, on voit que la distribution du groupe s'étale et se recontracte au fil du temps tout en gardant un pic qui est le groupe central.

In [ ]:
stats_oiseaux_2.plot()

On obtient les mêmes tendances de graphes que pour les *oiseaux_1*. Cela nous indique qu'il n'existe pas qu'une seule configuration possible pour une structure donnée. En faisant varier les paramètres, qu'on peut assimiler à des échelles de grandeur dans la réalité, on retrouve des structures connues à d'autres échelles.

## Essaim

Enfin, on va modéliser un essaim. Un essaim ne semble pas posséder de structure particulière. Les individus (des insectes la plupart du temps) sont très éloignés les uns des autres : ils possèdent de grandes zones d'attraction et de répulsion couplées à une zone d'orientation quasiment inexistante[$^{[6]}$](#Sources).

In [ ]:
N_essaim = N
num_steps_essaim = 400

essaim = Boids(N_essaim, cell_size=20, fov=15*np.pi/8, attraction=20., repulsion=20., orientation=2e-5,
               coherence_factor=.5, repulsion_factor=.5, align_factor=5e-10, vmax=5, vmin=3,
               bounds=100, bound_reaction=.4, spawn_size=100)

calcul(essaim, num_steps_essaim)

### Visualisation

In [ ]:
# matplotlib intéractif
%matplotlib notebook

anim = essaim.plot(num_steps_essaim, 150)

### Statistiques

In [ ]:
# plus besoin du mode interactif
%matplotlib inline

# calcul long car beaucoup de temps 
stats_essaim = Stats.from_history(essaim.get_history(), "insectes", num_steps_essaim)
stats_essaim.plot()

Comme les insectes sont très éloignés les uns des autres, on s'attend à ce qu'ils atteignent un maximum d'étalement, que leur alignement tendent vers un minimum et que les normes de la position, vitesse et accélération du barycentre tendent vers zéro, tous les déplacements et positions étant equiprobables, un peu comme un nuage d'atomes.
C'est bien ce qu'on obtient avec un étalement qui tend vers une occupation de tout l'espace disponible (dans ce cas là 200), et tous les autres paramètres (alignement et normes) qui tendent vers zéro et la distribution entre les boids s'étalant bien au fur et à mesure du temps.

## Conclusion

L'idée clé de l'algorithme des boids est de montrer que les animaux ayant des comportements grégaires n'ont pas besoin de communiquer entre eux afin de former des groupes organisés. Chaque boid individuellement fait simplement des choix par rapport à son entourage proche et, lorsque nous le faisons avec suffisement d'entre eux, nous voyons l'émergence de comportements similaires à ceux observés dans la nature.

Il est important de discuter des limitations de notre modèle. En effet, les animaux exhibent des comportements plus complexes que ceux que nous démontrons ici. Dans notre implémentation de l'algorithme des boids, ceux-ci n'intéragissent avec leur environnement qu'à travers les limites que nous avons imposées dans l'espace. Ceci est évidement une simplification de la réalité, dans laquelle des contraintes environnementales (météo, relief...) n'entrent pas en jeu.
    Dans notre étude, les boids ne communiquent pas entre eux mais utilisent seulement leur 'vision', alors qu'en réalité des études sur des poissons[$^{[10]}$](#Sources) ont démontré l'importance d'autre sens tels que celui de ressentir des changements de courants et de pression dans l'eau grâce à leur ligne latérale.
    
On peut souligner l'absence de certains comportements (destination préférentielle, fatigue, ...). En effet, nos boids ici n'ont pas de but précis. Nous avons également omis par soucis de simplicité la présence d'intéractions interspécifiques (par exemple avec un prédateur, ou d'autres espèces). Celles-ci auraient pu modifier leurs comportements (réaction de fuite, dispersion, attaque groupée face à une proie...).
    
Finalement, nous avons modélisé un groupe totalement homogène sans différences entre les individus (vitesse, force physique, âge, taille). Il arrive qu'il existe une hiérarchie au sein des groupes, c'est l'exemple de la "vache leadeuse" qui possède la meilleure fonction de relation (meilleure vue, meilleure force physique), qui va alors guider le troupeau dans sa recherche de nourriture.

## Sources
1. [Comportment grégaire](https://fr.wikipedia.org/wiki/Comportement_gr%C3%A9gaire)
2. [Reynolds, C.W. (1987). "Flocks, Herds, and Schools: A Distributed Behavioral Model". _Computer Graphics_, __21__(4), pp.25-34.](https://team.inria.fr/imagine/files/2014/10/flocks-hers-and-schools.pdf)
3. [Flock Defense](https://web.stanford.edu/group/stanfordbirds/text/essays/Flock_Defense.html)
4. [Site de Craig W. Reynolds](http://www.red3d.com/cwr/boids/)
5. [Pseudocode boids](http://www.kfish.org/boids/pseudocode.html)
6. [Boids Wikipédia](https://fr.wikipedia.org/wiki/Boids)
7. [Taille d'une nuée](https://support.ebird.org/en/support/solutions/articles/48000838845-how-to-count-birds)
8. [Spatial hash](http://mauveweb.co.uk/posts/2011/05/introduction-to-spatial-hashes.html)
9. [Paramètres oiseaux](http://people.ece.cornell.edu/land/courses/ece4760/labs/s2021/Boids/Boids.html)
10. [Pitcher, T. J., Partridge, B. L., Wardle, C. S., "Blind Fish Can School," Science 194,#4268 (1976), p. 964.](https://www.science.org/doi/10.1126/science.982056)